# Import the data:
that was saved from data_generation.ipynb

In [ ]:
!pip install matplotlib dill

In [ ]:
import dill as pickle

try:
    with open("model_answers_fixed.pkl", "rb") as f:
        loaded_model_answers = pickle.load(f)

    print("✅ Successfully loaded the fixed pickle file!")
    print(f"Loaded object type: {type(loaded_model_answers)}")  # Should print <class 'dict'>
except Exception as e:
    print("⚠️ Error while loading:", e)


In [ ]:
import dill as pickle

try:
    with open("model_answers_fixed_two.pkl", "rb") as f:
        loaded_model_answers2 = pickle.load(f)

    print("✅ Successfully loaded the fixed pickle file!")
    print(f"Loaded object type: {type(loaded_model_answers)}")  # Should print <class 'dict'>
except Exception as e:
    print("⚠️ Error while loading:", e)

In [ ]:
loaded_model_answers.keys()

In [ ]:
loaded_model_answers2[0]

In [ ]:
def merge_dictionaries_incrementing_keys(dict1, dict2):
    """Merges two dictionaries, incrementing keys of the second dictionary to prevent overlap."""
    merged_dict = dict1.copy()  # Start with the first dictionary
    
    # Find the highest existing key in dict1
    max_key = max(dict1.keys(), default=-1) + 1  

    # Add entries from dict2, incrementing keys
    for key, value in dict2.items():
        merged_dict[max_key] = value
        max_key += 1  # Ensure unique keys

    return merged_dict


# Merge with incremented indices
merged_model_answers = merge_dictionaries_incrementing_keys(loaded_model_answers, loaded_model_answers2)


In [ ]:
merged_model_answers.keys()

# Separate the correct and incorrect paths
Note there is some randomness here because I deliberately only select one path (out of 3) for each question, and I select a random incorrect path
I basically give the model 3 tries to get it right and use a parsing library that sometimes gets it wrong, so I bias towards high-confidence incorrect examples and correct examples that the parser found in at least one try.

In [ ]:
import random

correct_paths = []
incorrect_paths = []

# Iterate through each key-value pair
for key, value_dict in merged_model_answers.items():
    try:
        # Ensure value_dict is a dictionary and contains `correctness_list`
        if isinstance(value_dict, dict) and 'correctness_list' in value_dict:
            correctness_list = value_dict['correctness_list']
            correct_indices = [idx for idx, correctness in enumerate(correctness_list) if correctness == 1]

            if correct_indices:
                # Pick the first correct path
                idx = correct_indices[0]
                correct_paths.append(value_dict[idx].get('raw_model_answer', ''))
            else:
                # If all are incorrect, pick a random incorrect path
                incorrect_indices = [idx for idx in range(len(correctness_list)) if idx in value_dict]
                if incorrect_indices:
                    idx = random.choice(incorrect_indices)
                    incorrect_paths.append(value_dict[idx].get('raw_model_answer', ''))

    except Exception as e:
        print(f"⚠️ Skipping key {key}: Could not process value. Error: {e}")

# Display results
print("Correct Paths:", correct_paths[:3])  # Print first 3 for verification
print("Incorrect Paths:", incorrect_paths[:3])  # Print first 3 for verification

print("Correct rate: ",len(correct_paths)/(len(correct_paths) + len(incorrect_paths)))
print("Correct rate: ",len(incorrect_paths)/(len(correct_paths) + len(incorrect_paths)))


In [ ]:

print(len(correct_paths))
print(len(incorrect_paths))

In [ ]:
!pip install sentence_transformers

# Lets split our incorrect and correct paths into thoughts and compute iterative embeddings for each thought

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def split_into_thoughts(paths, delimiter="\n"):
    """
    Splits each CoT response into a list of thoughts based on the given delimiter.
    
    Parameters:
    - paths (list of str): Each entry is a CoT response as a single string.

    Returns:
    - list of list of str: Each CoT response split into separate thoughts.
    """
    return [path.split(delimiter) for path in paths]

def compute_thoughtwise_embedding_differences_aggregated(reasoning_paths):
    """
    Computes thought-to-thought embedding similarity across multiple completions,
    while aggregating results across different questions.

    Parameters:
    - reasoning_paths (list of list of str): Each entry is a CoT path, which is a list of thought.

    Returns:
    - mean_similarities (list of float): Mean thoughtwise thought-to-thought similarity scores.
    - std_similarities (list of float): Standard deviation of thoughtwise similarities (for variability analysis).
    """
    if not reasoning_paths:
        print("⚠️ Warning: Received an empty list of reasoning paths.")
        return [], []

    max_thoughts = max(len(path) for path in reasoning_paths)
    all_similarities = [[] for _ in range(max_thoughts - 1)]  # Store similarities for each thought index

    print(f"Debugging: Max Thoughts Across Paths: {max_thoughts}")
    print(f"Debugging: Number of Reasoning Paths: {len(reasoning_paths)}")

    for path_idx, path in enumerate(reasoning_paths):
        print(f"\nProcessing path {path_idx + 1}/{len(reasoning_paths)} with {len(path)} thoughts")


        for thought in range(1, len(path)):  # Start at thought 1 to compare with thought 0
            prev_thought_text = path[thought - 1].strip()
            curr_thought_text = path[thought].strip()

            if not prev_thought_text or not curr_thought_text:
                #print(f"⚠️ thought {thought}: Skipping empty thought in path {path_idx + 1}")
                continue  # Skip empty thoughts

            # Compute embeddings
            prev_thought_embedding = embedder.encode(prev_thought_text, convert_to_tensor=True)
            curr_thought_embedding = embedder.encode(curr_thought_text, convert_to_tensor=True)

            # Compute cosine similarity
            similarity = torch.nn.functional.cosine_similarity(prev_thought_embedding, curr_thought_embedding, dim=0).item()
            all_similarities[thought - 1].append(similarity)

    # Aggregate mean and std similarity across different paths for each thought index
    mean_similarities = [np.mean(sim) if sim else 0 for sim in all_similarities]
    std_similarities = [np.std(sim) if sim else 0 for sim in all_similarities]

    print(f"Debugging: Computed {len(mean_similarities)} mean similarities and {len(std_similarities)} standard deviations")

    return mean_similarities, std_similarities

# Example reasoning paths (for debugging)


# Split responses into thoughtwise reasoning paths
correct_paths_split = split_into_thoughts(correct_paths)
wrong_paths_split = split_into_thoughts(incorrect_paths)

# Compute aggregated thoughtwise similarity differences for correct and wrong CoTs
mean_thoughtwise_correct, std_thoughtwise_correct = compute_thoughtwise_embedding_differences_aggregated(correct_paths_split)
mean_thoughtwise_wrong, std_thoughtwise_wrong = compute_thoughtwise_embedding_differences_aggregated(wrong_paths_split)

# Debugging: Check if similarity lists have unexpected lengths
print(f"Debugging: Length of Correct Similarities: {len(mean_thoughtwise_correct)}")
print(f"Debugging: Length of Wrong Similarities: {len(mean_thoughtwise_wrong)}")




In [ ]:
# 📊 **Plot thoughtwise similarity**
if mean_thoughtwise_correct and mean_thoughtwise_wrong:
    plt.figure(figsize=(10, 5))

    plt.plot(range(1, len(mean_thoughtwise_correct) + 1), mean_thoughtwise_correct, label="Correct Paths", color="green")
    plt.fill_between(range(1, len(mean_thoughtwise_correct) + 1), 
                     np.array(mean_thoughtwise_correct) - np.array(std_thoughtwise_correct),
                     np.array(mean_thoughtwise_correct) + np.array(std_thoughtwise_correct),
                     color="green", alpha=0.2)

    plt.plot(range(1, len(mean_thoughtwise_wrong) + 1), mean_thoughtwise_wrong, label="Wrong Paths", color="red")
    plt.fill_between(range(1, len(mean_thoughtwise_wrong) + 1), 
                     np.array(mean_thoughtwise_wrong) - np.array(std_thoughtwise_wrong),
                     np.array(mean_thoughtwise_wrong) + np.array(std_thoughtwise_wrong),
                     color="red", alpha=0.2)

    plt.xlabel("Thought Number")
    plt.ylabel("Thought-to-Thought Cosine Similarity")
    plt.title("Thought-to-Thought Similarity: Correct vs. Wrong Paths (Aggregated)")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("⚠️ Warning: Thoughtwise similarity data is empty, cannot plot results.")

In [ ]:
!pip install datasets

In [ ]:
# Load MATH-500 dataset
from datasets import load_dataset


print("📥 Loading MATH-500 dataset...")
dataset = load_dataset("HuggingFaceH4/MATH-500")

# In the cell below I compute the paths grouped by difficulty

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import defaultdict

# Extract problem difficulty levels (1 to 5)
problem_difficulties = dataset['test']['level']

# Initialize dictionaries for correct and incorrect paths grouped by difficulty
paths_by_difficulty_correct = {level: [] for level in range(1, 6)}
paths_by_difficulty_wrong = {level: [] for level in range(1, 6)}

# Populate correct and incorrect paths ensuring **only one per item**
for key, value_dict in merged_model_answers.items():
    try:
        if isinstance(value_dict, dict) and 'correctness_list' in value_dict:
            correctness_list = value_dict['correctness_list']

            if key >= len(problem_difficulties):  
                print(f"⚠️ Skipping key {key}: No matching difficulty level found.")
                continue

            difficulty_level = problem_difficulties[key]  # Get corresponding difficulty

            # Find correct paths
            correct_indices = [idx for idx, correctness in enumerate(correctness_list) if correctness == 1]

            if correct_indices:
                # Pick only the first correct path
                idx = correct_indices[0]
                raw_model_answer = value_dict[idx].get('raw_model_answer', '')
                paths_by_difficulty_correct[difficulty_level].append(raw_model_answer)
            else:
                # If all are incorrect, select one random incorrect path
                incorrect_indices = [idx for idx in range(len(correctness_list)) if idx in value_dict]
                if incorrect_indices:
                    idx = random.choice(incorrect_indices)
                    raw_model_answer = value_dict[idx].get('raw_model_answer', '')
                    paths_by_difficulty_wrong[difficulty_level].append(raw_model_answer)

    except Exception as e:
        print(f"⚠️ Skipping key {key}: Could not process value. Error: {e}")


print(paths_by_difficulty_correct)
print(paths_by_difficulty_wrong)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example data (replace with actual counts from your dataset)
difficulty_levels = [1, 2, 3, 4, 5]
correct_counts = [len(paths_by_difficulty_correct[level]) for level in difficulty_levels]
incorrect_counts = [len(paths_by_difficulty_wrong[level]) for level in difficulty_levels]

# Set up bar width and positions
bar_width = 0.35
x = np.arange(len(difficulty_levels))

# Create the bar chart
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - bar_width/2, correct_counts, bar_width, label='Correct', color='green')
ax.bar(x + bar_width/2, incorrect_counts, bar_width, label='Incorrect', color='red')

# Labels and formatting
ax.set_xlabel('Difficulty Level')
ax.set_ylabel('Number of Answers')
ax.set_title('Number of Correct and Incorrect Answers per Difficulty Level')
ax.set_xticks(x)
ax.set_xticklabels(difficulty_levels)
ax.legend()

# Show the plot
plt.show()


# In the cell below I look for any interesting trends regarding the path length or branching factor
- Didnt find anything interesting here worth reporting, the average path length is longer for incorrect rather than correct solutions but this is trivial

In [ ]:

# Compute Path Lengths and Branching Factors**
def compute_path_lengths(paths):
    """Computes the number of steps in each path."""
    return [len(path) for path in paths]

def compute_branching_factor(paths):
    """Computes the unique step transitions per reasoning step."""
    max_steps = max(len(path) for path in paths) if paths else 0
    stepwise_transitions = [defaultdict(int) for _ in range(max_steps)]

    for path in paths:
        for step_idx in range(len(path) - 1):  # Skip last step
            transition = (path[step_idx], path[step_idx + 1])
            stepwise_transitions[step_idx][transition] += 1

    return [len(step.keys()) for step in stepwise_transitions if step]

# Compute Metrics Across Difficulty Levels**
difficulty_path_lengths_correct = {}
difficulty_path_lengths_wrong = {}
difficulty_branching_factors_correct = {}
difficulty_branching_factors_wrong = {}

for level in range(1, 6):  # Loop over difficulty levels 1 to 5
    if paths_by_difficulty_correct[level]:
        difficulty_path_lengths_correct[level] = compute_path_lengths(paths_by_difficulty_correct[level])
        difficulty_branching_factors_correct[level] = compute_branching_factor(paths_by_difficulty_correct[level])
    else:
        difficulty_path_lengths_correct[level] = []


    if paths_by_difficulty_wrong[level]:
        difficulty_path_lengths_wrong[level] = compute_path_lengths(paths_by_difficulty_wrong[level])
        difficulty_branching_factors_wrong[level] = compute_branching_factor(paths_by_difficulty_wrong[level])
    else:
        difficulty_path_lengths_wrong[level] = []
        difficulty_branching_factors_wrong[level] = []
        
# Visualization**
plt.figure(figsize=(12, 5))

# **1️⃣ Path Lengths vs. Difficulty (Correct vs. Incorrect)**
plt.subplot(1, 2, 1)
mean_lengths_correct = [np.mean(difficulty_path_lengths_correct[level]) if difficulty_path_lengths_correct[level] else 0 for level in range(1, 6)]
mean_lengths_wrong = [np.mean(difficulty_path_lengths_wrong[level]) if difficulty_path_lengths_wrong[level] else 0 for level in range(1, 6)]

plt.plot(range(1, 6), mean_lengths_correct, marker="o", linestyle="-", color="green", label="Correct Paths")
plt.plot(range(1, 6), mean_lengths_wrong, marker="o", linestyle="-", color="red", label="Incorrect Paths")

plt.xlabel("Problem Difficulty (1 = Easy, 5 = Hard)")
plt.ylabel("Avg. Path Length (Steps)")
plt.title("Path Length vs. Problem Difficulty (Correct vs. Incorrect)")
plt.grid(True)
plt.legend()

# **2️⃣ Branching Factor vs. Difficulty (Correct vs. Incorrect)**
plt.subplot(1, 2, 2)
mean_branching_correct = [np.mean(difficulty_branching_factors_correct[level]) if difficulty_branching_factors_correct[level] else 0 for level in range(1, 6)]
mean_branching_wrong = [np.mean(difficulty_branching_factors_wrong[level]) if difficulty_branching_factors_wrong[level] else 0 for level in range(1, 6)]

plt.plot(range(1, 6), mean_branching_correct, marker="o", linestyle="-", color="green", label="Correct Paths")
plt.plot(range(1, 6), mean_branching_wrong, marker="o", linestyle="-", color="red", label="Incorrect Paths")

plt.xlabel("Problem Difficulty (1 = Easy, 5 = Hard)")
plt.ylabel("Avg. Branching Factor")
plt.title("Branching Factor vs. Problem Difficulty (Correct vs. Incorrect)")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# In the cells below I compute what I did above but split for each difficulty
There are no intersting trends and a bigram word model seems to simplistic to use as a measure compared to embeddings or entropy, these results were excluded from the report

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import defaultdict

def split_into_steps(path):
    """Splits a path into individual steps (assumes space-separated steps)."""
    return path.split()

def compute_path_lengths(paths):
    """Computes the length of each path (number of steps)."""
    return [len(path) for path in paths]

def remove_outliers(data):
    """Removes outliers using the Interquartile Range (IQR) method."""
    if not data:
        return data  # Return empty list if no data is provided
    
    q1 = np.percentile(data, 25)  # First quartile (Q1)
    q3 = np.percentile(data, 75)  # Third quartile (Q3)
    iqr = q3 - q1  # Interquartile range
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    return [x for x in data if lower_bound <= x <= upper_bound]

def compute_branching_factor(paths):
    """Computes the average number of unique step transitions at each step."""
    if not paths:
        return []
    
    max_steps = max(len(path) for path in paths)
    stepwise_transitions = [defaultdict(int) for _ in range(max_steps)]

    # Count unique transitions at each step
    for path in paths:
        for step_idx in range(len(path) - 1):  # Skip last step
            transition = (path[step_idx], path[step_idx + 1])
            stepwise_transitions[step_idx][transition] += 1

    # Compute branching factor per step
    return [len(step.keys()) for step in stepwise_transitions if step]

# **1️⃣ Organize Paths by Difficulty Level**
difficulty_levels = dataset['test']['level']  # Extract problem difficulties

# Create separate dictionaries for correct and incorrect paths per difficulty level
correct_paths_by_difficulty = {level: [] for level in range(1, 6)}
wrong_paths_by_difficulty = {level: [] for level in range(1, 6)}

# Assign one path per difficulty level
for key, value_dict in merged_model_answers.items():
    try:
        if isinstance(value_dict, dict) and 'correctness_list' in value_dict:
            correctness_list = value_dict['correctness_list']

            if key >= len(difficulty_levels):
                print(f"⚠️ Skipping key {key}: No matching difficulty level found.")
                continue
            
            difficulty_level = difficulty_levels[key]  # Get corresponding difficulty level
            correct_indices = [idx for idx, correctness in enumerate(correctness_list) if correctness == 1]

            if correct_indices:
                # Pick the first correct path and split into steps
                idx = correct_indices[0]
                path = value_dict[idx].get('raw_model_answer', '')
                correct_paths_by_difficulty[difficulty_level].append(split_into_steps(path))
            else:
                # If all are incorrect, pick a random incorrect path and split into steps
                incorrect_indices = [idx for idx in range(len(correctness_list)) if idx in value_dict]
                if incorrect_indices:
                    idx = random.choice(incorrect_indices)
                    path = value_dict[idx].get('raw_model_answer', '')
                    wrong_paths_by_difficulty[difficulty_level].append(split_into_steps(path))

    except Exception as e:
        print(f"⚠️ Skipping key {key}: Could not process value. Error: {e}")

# **2️⃣ Compute Path Lengths and Branching Factors for Each Difficulty**
filtered_path_lengths_correct = {}
filtered_path_lengths_wrong = {}
filtered_branching_factors_correct = {}
filtered_branching_factors_wrong = {}

for level in range(1, 6):
    if correct_paths_by_difficulty[level]:
        raw_lengths = compute_path_lengths(correct_paths_by_difficulty[level])
        filtered_path_lengths_correct[level] = remove_outliers(raw_lengths)
        filtered_branching_factors_correct[level] = compute_branching_factor(
            [path for path in correct_paths_by_difficulty[level] if len(path) in filtered_path_lengths_correct[level]]
        )
    else:
        filtered_path_lengths_correct[level] = []
        filtered_branching_factors_correct[level] = []

    if wrong_paths_by_difficulty[level]:
        raw_lengths = compute_path_lengths(wrong_paths_by_difficulty[level])
        filtered_path_lengths_wrong[level] = remove_outliers(raw_lengths)
        filtered_branching_factors_wrong[level] = compute_branching_factor(
            [path for path in wrong_paths_by_difficulty[level] if len(path) in filtered_path_lengths_wrong[level]]
        )
    else:
        filtered_path_lengths_wrong[level] = []
        filtered_branching_factors_wrong[level] = []

# **3️⃣ Visualization**
for level in range(1, 6):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # **1️⃣ Path Length Distribution for this Difficulty Level**
    axes[0].hist(filtered_path_lengths_correct[level], bins=20, alpha=0.5, label="Correct", color="blue")
    axes[0].hist(filtered_path_lengths_wrong[level], bins=20, alpha=0.5, label="Incorrect", color="red", linestyle="dashed")

    axes[0].set_xlabel("Path Length (Number of Words)")
    axes[0].set_ylabel("Frequency")
    axes[0].set_title(f"Path Length Distribution - Level {level}")
    axes[0].legend()

    # **2️⃣ Stepwise Branching Factor Trends for this Difficulty Level**
    axes[1].plot(range(1, len(filtered_branching_factors_correct[level]) + 1), 
                 filtered_branching_factors_correct[level], 
                 marker="o", linestyle="-", label="Correct", color="blue")

    axes[1].plot(range(1, len(filtered_branching_factors_wrong[level]) + 1), 
                 filtered_branching_factors_wrong[level], 
                 marker="x", linestyle="--", label="Incorrect", color="red")

    axes[1].set_xlabel("Word Index")
    axes[1].set_ylabel("Branching Factor (Unique Transitions)")
    axes[1].set_title(f"Wordwise Branching Factor Trends - Level {level}")
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
!pip install seaborn umap-learn numpy==2.1

# At this point I wanted to try clustering techniques on the embeddings to see if I could get some interesting separation. In all fairness, I tried PCA+Kmeans, with no luck.
However, ChatGpt recommended AgglomerativeClustering for the following reason:

UMAP + Agglomerative Clustering more effectively captures non-linear reasoning patterns in Chain-of-Thought (CoT) analysis than PCA + K-Means. UMAP preserves local structure, maintaining similarity between correct and incorrect reasoning paths, while PCA’s linear assumptions may obscure key distinctions. Agglomerative Clustering adapts to hierarchical reasoning trajectories, distinguishing between premature convergence (closely clustered incorrect paths) and excessive exploration (widely dispersed incorrect paths), whereas K-Means imposes rigid clusters that may not align with natural failure modes.

By revealing both local and global structure, UMAP + Agglomerative Clustering offers a more interpretable and flexible analysis of reasoning failures in language models.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def get_embeddings(paths):
    """Generates sentence embeddings for a list of paths."""
    if not paths:
        return np.array([])  # Return empty array if no paths available
    
    return np.array(embedder.encode(paths, convert_to_numpy=True))

# Generate Embeddings for Correct and Incorrect Paths by Difficulty**
correct_embeddings_by_difficulty = {}
wrong_embeddings_by_difficulty = {}

for level in range(1, 6):
    correct_embeddings_by_difficulty[level] = get_embeddings(correct_paths_by_difficulty[level])
    wrong_embeddings_by_difficulty[level] = get_embeddings(wrong_paths_by_difficulty[level])

# Apply UMAP and Hierarchical Clustering for Each Difficulty Level**
fig, axes = plt.subplots(2, 3, figsize=(15, 10))  # 2 Rows, 3 Columns
axes = axes.flatten()

for level in range(1, 6):
    ax = axes[level - 1]  # Select subplot
    
    # Combine embeddings for clustering
    embeddings = []
    correctness_labels = []

    if correct_embeddings_by_difficulty[level].size > 0:
        embeddings.append(correct_embeddings_by_difficulty[level])
        correctness_labels.extend(["Correct"] * len(correct_embeddings_by_difficulty[level]))

    if wrong_embeddings_by_difficulty[level].size > 0:
        embeddings.append(wrong_embeddings_by_difficulty[level])
        correctness_labels.extend(["Incorrect"] * len(wrong_embeddings_by_difficulty[level]))

    if embeddings:
        # Stack embeddings
        embeddings = np.vstack(embeddings)
        
        # Apply UMAP**
        umap_reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, metric='cosine')
        reduced_embeddings = umap_reducer.fit_transform(embeddings)

        # Agglomerative Clustering**
        clustering = AgglomerativeClustering(n_clusters=2)  # Try different numbers of clusters
        cluster_labels = clustering.fit_predict(reduced_embeddings)

        # Plot Clusters**
        sns.scatterplot(x=reduced_embeddings[:, 0], y=reduced_embeddings[:, 1], hue=cluster_labels, palette="deep", ax=ax)

        ax.set_xlabel("UMAP Component 1")
        ax.set_ylabel("UMAP Component 2")
        ax.set_title(f"UMAP + Hierarchical Clustering (Difficulty {level})")

# Remove last empty subplot (for 2x3 layout)
fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import umap
from scipy.spatial.distance import cdist
from sentence_transformers import SentenceTransformer

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def get_embeddings(paths):
    """Generates embeddings for a list of paths."""
    if not paths:
        return np.array([])  # Return empty array if no paths available
    return np.array(embedder.encode(paths, convert_to_numpy=True))

# **1️⃣ Generate Embeddings for Correct and Incorrect Paths by Difficulty**
correct_embeddings_by_difficulty = {}
wrong_embeddings_by_difficulty = {}

for level in range(1, 6):
    correct_embeddings_by_difficulty[level] = get_embeddings(correct_paths_by_difficulty[level])
    wrong_embeddings_by_difficulty[level] = get_embeddings(wrong_paths_by_difficulty[level])

# **2️⃣ Apply UMAP and Find Most Distant Correct-Incorrect Pairs**
distant_examples = {}

for level in range(1, 6):
    correct_embeddings = correct_embeddings_by_difficulty[level]
    wrong_embeddings = wrong_embeddings_by_difficulty[level]
    correct_paths = correct_paths_by_difficulty[level]
    wrong_paths = wrong_paths_by_difficulty[level]

    if correct_embeddings.size > 0 and wrong_embeddings.size > 0:
        # Stack embeddings separately for correct and incorrect paths
        combined_embeddings = np.vstack([correct_embeddings, wrong_embeddings])
        combined_labels = ["Correct"] * len(correct_embeddings) + ["Incorrect"] * len(wrong_embeddings)
        combined_paths = correct_paths + wrong_paths  # Keep corresponding text paths

        # **Apply UMAP**
        umap_reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, metric='cosine')
        reduced_embeddings = umap_reducer.fit_transform(combined_embeddings)

        # **Separate Correct and Incorrect UMAP-Reduced Embeddings**
        correct_indices = np.array([i for i, label in enumerate(combined_labels) if label == "Correct"])
        incorrect_indices = np.array([i for i, label in enumerate(combined_labels) if label == "Incorrect"])

        reduced_correct = reduced_embeddings[correct_indices]
        reduced_incorrect = reduced_embeddings[incorrect_indices]

        # **Compute Pairwise Distances Between Correct and Incorrect Embeddings**
        distance_matrix = cdist(reduced_correct, reduced_incorrect, metric="euclidean")

        # **Find the Most Distant Correct-Incorrect Pair**
        i, j = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
        correct_index = correct_indices[i]
        incorrect_index = incorrect_indices[j]

        distant_examples[level] = (combined_paths[correct_index], combined_paths[incorrect_index])

# **3️⃣ Print the Most Distant Correct vs. Incorrect Paths for Each Difficulty Level**
for level, (correct_path, incorrect_path) in distant_examples.items():
    print(f"\n### Difficulty {level} - Most Distant Correct vs. Incorrect Paths ###")
    print("\nCorrect Example:\n", correct_path)
    print("\nIncorrect Example:\n", incorrect_path)
    print("-" * 80)


The distance in embedding space can be explained by differences in problem type, reasoning structure, and clarity between the correct and incorrect paths.

### 1 Different problem types
Example (Difficulty 1):
Correct Path: Computes the modular inverse of 997 mod 1000, using the Extended Euclidean Algorithm.
Incorrect Path: Solves a completely different algebraic problem involving systems of equations with three variables.
Why They're Far Apart: These problems require entirely different mathematical approaches—modular arithmetic vs. polynomial equation solving—so embeddings naturally separate them.

### 2️ Stepwise vs. Exploratory Reasoning
Example (Difficulty 3):
Correct Path: Methodically applies trigonometric identities to find tan(A) in a right triangle.
Incorrect Path: Attempts to simplify a sum involving roots of unity, but instead of following a structured method, the reasoning jumps around leading to confusion.
Why They're Far Apart: The correct path maintains logical consistency, while the incorrect path lacks focus, leading to higher entropy in reasoning flow.

### 3 Lexical & Conceptual Drift (Precision vs. Redundant Exploration)
Example (Difficulty 2):
Correct Path: Uses geometric properties (Thales' theorem) to determine the measure of an angle in an inscribed triangle.
Incorrect Path: Attempts to rotate a complex number 90 degrees, an unrelated problem with a different mathematical framework.
Why They're Far Apart: Even though both involve geometry, the incorrect path introduces conceptually unrelated transformations, shifting its embedding further away from the structured solution.
Conclusion

The large distance in embedding space arises because:
- Correct and incorrect paths often solve entirely different problems.
- Correct paths follow structured, stepwise reasoning, while incorrect paths are more erratic.
- Incorrect responses introduce unrelated concepts, causing conceptual drift in embeddings.

Intuition: Since embeddings encode semantic and logical coherence, incorrect solutions that introduce topic shifts, redundant steps, or unrelated concepts naturally map further from structured, correct responses.







# This is a bit naive but I want to compare the embeddings of incorrect and correct responses side by side to see if I can see any patterns:
- the prediction is that correct might be tighter together

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def get_embeddings(paths):
    """Generates sentence embeddings for a list of paths."""
    if not paths:
        return np.array([])  
    return np.array(embedder.encode(paths, convert_to_numpy=True))

# Generate Embeddings for Correct and Incorrect Paths**
correct_embeddings_by_difficulty = {}
wrong_embeddings_by_difficulty = {}

for level in range(1, 6):
    correct_embeddings_by_difficulty[level] = get_embeddings(correct_paths_by_difficulty[level])
    wrong_embeddings_by_difficulty[level] = get_embeddings(wrong_paths_by_difficulty[level])

# Apply UMAP and Agglomerative Clustering for Both Correct & Incorrect Paths**
fig, axes = plt.subplots(2, 5, figsize=(20, 10))  # 2 Rows (Correct & Incorrect), 5 Columns (Difficulty Levels)

clustered_responses = {
    "correct": {},
    "incorrect": {}
}

for level in range(1, 6):
    for category, embeddings_dict, paths_dict, row_idx in [
        ("correct", correct_embeddings_by_difficulty, correct_paths_by_difficulty, 0),
        ("incorrect", wrong_embeddings_by_difficulty, wrong_paths_by_difficulty, 1),
    ]:
        ax = axes[row_idx, level - 1]  # Select subplot
        paths = paths_dict[level]
        embeddings = embeddings_dict[level]

        if embeddings.size > 0:
            # **Apply UMAP**
            umap_reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, metric='cosine')
            reduced_embeddings = umap_reducer.fit_transform(embeddings)

            # **Apply Agglomerative Clustering**
            clustering = AgglomerativeClustering(n_clusters=3)
            cluster_labels = clustering.fit_predict(reduced_embeddings)

            # Store responses by cluster
            clustered_responses[category][level] = {i: [] for i in range(3)}
            for idx, label in enumerate(cluster_labels):
                clustered_responses[category][level][label].append(paths[idx])

            # **Plot Clusters**
            sns.scatterplot(
                x=reduced_embeddings[:, 0], 
                y=reduced_embeddings[:, 1], 
                hue=cluster_labels,  
                palette="deep",  
                ax=ax, alpha=0.7, s=50  
            )

            ax.set_xlabel("UMAP Component 1")
            ax.set_ylabel("UMAP Component 2")
            ax.set_title(f"{category.capitalize()} Paths - Difficulty {level}")

# Adjust layout
plt.tight_layout()
plt.show()

# Print Clustered Responses for Both Correct & Incorrect Paths**
for category in ["correct", "incorrect"]:
    print(f"\n### Clustered {category.capitalize()} Responses ###\n")
    for level, clusters in clustered_responses[category].items():
        print(f"\n### Difficulty {level} ###\n")
        for cluster_id, responses in clusters.items():
            print(f"\n--- Cluster {cluster_id} ---\n")
            for response in responses[:3]:  # Print up to 3 examples per cluster
                print(response)
                print("-" * 80)


# I got an idea to see what the entropy does for each word in the CoT after thinking about the entropy for clusters in the UMAP plot, so I went ahead and looked at the word wise entropy split by difficulty

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from scipy.stats import entropy
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def get_embeddings(paths):
    """Generates embeddings for a list of paths."""
    if not paths:
        return np.array([])  # Return empty array if no paths available
    return np.array(embedder.encode(paths, convert_to_numpy=True))

def compute_wordwise_entropy(paths):
    """
    Computes entropy at each word of CoT to measure randomness in reasoning.

    Parameters:
    - paths (list of list of str): Multiple CoT responses split into words.

    Returns:
    - List of entropy values per word.
    """
    max_words = max(len(path) for path in paths)
    wordwise_distributions = [{} for _ in range(max_words)]

    for path in paths:
        for word_idx, word in enumerate(path):
            wordwise_distributions[word_idx][word] = wordwise_distributions[word_idx].get(word, 0) + 1

    entropy_scores = []
    for word_dist in wordwise_distributions:
        probs = np.array(list(word_dist.values())) / sum(word_dist.values()) if word_dist else np.array([1])
        entropy_scores.append(entropy(probs))

    return entropy_scores  # Return entropy per word

def remove_word_outliers(word_counts):
    """
    Removes outliers from word numbers using the IQR method.

    Parameters:
    - word_counts (list of int): List of word numbers.

    Returns:
    - int: Upper bound for word numbers after removing outliers.
    """
    q1 = np.percentile(word_counts, 25)
    q3 = np.percentile(word_counts, 75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr  # Define outlier threshold

    return int(min(upper_bound, max(word_counts)))  # Ensure within max word range

# Generate Embeddings for Correct and Incorrect Paths by Difficulty**
correct_embeddings_by_difficulty = {}
wrong_embeddings_by_difficulty = {}
correct_wordwise_entropy = {}
wrong_wordwise_entropy = {}

word_lengths = []  # Track word lengths to filter outliers

for level in range(1, 6):
    correct_paths = correct_paths_by_difficulty[level]
    wrong_paths = wrong_paths_by_difficulty[level]

    correct_embeddings_by_difficulty[level] = get_embeddings(correct_paths)
    wrong_embeddings_by_difficulty[level] = get_embeddings(wrong_paths)

    # Compute wordwise entropy for correct and incorrect paths
    correct_wordwise_entropy[level] = compute_wordwise_entropy(correct_paths)
    wrong_wordwise_entropy[level] = compute_wordwise_entropy(wrong_paths)

    # Track word lengths for outlier removal
    word_lengths.extend([len(path) for path in correct_paths])
    word_lengths.extend([len(path) for path in wrong_paths])

# Remove Outliers from word Number (X-Axis)**
max_valid_words = remove_word_outliers(word_lengths)

# Create Separate wordwise Entropy Plots Per Difficulty Level**
fig, axes = plt.subplots(2, 3, figsize=(15, 10))  # 2 Rows, 3 Columns
axes = axes.flatten()

for level in range(1, 6):
    ax = axes[level - 1]  # Select subplot

    if level in correct_wordwise_entropy and level in wrong_wordwise_entropy:
        correct_entropy_per_word = correct_wordwise_entropy[level][:max_valid_words]
        incorrect_entropy_per_word = wrong_wordwise_entropy[level][:max_valid_words]

        ax.plot(range(1, len(correct_entropy_per_word) + 1), correct_entropy_per_word, linestyle="-", color="green", label="Correct Paths")
        ax.plot(range(1, len(incorrect_entropy_per_word) + 1), incorrect_entropy_per_word, linestyle="--", color="red", label="Incorrect Paths")

        ax.set_xlabel("Word Index")
        ax.set_ylabel("Entropy (Wordwise Randomness)")
        ax.set_title(f"Wordwise Entropy Trends (Difficulty {level})")
        ax.grid(True)
        ax.legend()

# Remove last empty subplot (for 2x3 layout)
fig.delaxes(axes[-1])

plt.tight_layout()
plt.xlim(1, max_valid_words)
plt.show()
